**패키지 설치**

In [ ]:
# Python
!pip install python-dotenv
# 서버 관련
!pip install flask flask-cors pyngrok
!pip install flask-ngrok
!pip install pyngrok
# AI 관련
!pip install ftfy regex tqdm
!pip install controlnet_aux
!pip install diffusers transformers accelerate

**초기 환경 설정**

 - Google Drive 마운트

 - 환경 변수 로드

In [ ]:
from google.colab import drive
from dotenv import load_dotenv

drive.mount('/content/drive')
load_dotenv('/content/drive/MyDrive/Colab Notebooks/Pinger/.env')

Mounted at /content/drive


True

**이미지 생성 AI**

In [ ]:
import base64
import io
from PIL import Image
import torch
from datetime import datetime

from diffusers import ControlNetModel, StableDiffusionControlNetPipeline, UniPCMultistepScheduler
from controlnet_aux import HEDdetector

class ImageAI:
    def __init__(self):
        self.hed = HEDdetector.from_pretrained('lllyasviel/Annotators')

        # ControlNet Scribble 모델 로드
        self.controlnet = ControlNetModel.from_pretrained(
            "lllyasviel/sd-controlnet-scribble",
            torch_dtype=torch.float16
        )

        # Stable Diffusion 파이프라인 설정
        self.pipe = StableDiffusionControlNetPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            controlnet=self.controlnet,
            safety_checker=None,
            torch_dtype=torch.float16
        )

        self.pipe.scheduler = UniPCMultistepScheduler.from_config(self.pipe.scheduler.config)
        self.pipe.enable_model_cpu_offload()
    ## __init__(self)

    def generate_from_sketch(self, b64_string: str, prompt: str) -> str:
        init_image = self.__base64_to_pil(b64_string)

        detected_scribble = self.hed(init_image) # 선 감지

        output_image = self.pipe(
            prompt=prompt,
            image=detected_scribble,
            guidance_scale=7.5,
            num_inference_steps=30
        ).images[0]

        self.__save_to_drive(output_image, prompt)

        return self.__pil_to_base64(output_image)
    ## generate_from_sketch(self, b64_string: str, prompt: str)

    @staticmethod
    def __save_to_drive(img: Image.Image, prompt: str):
        save_dir = "/content/drive/MyDrive/"
        os.makedirs(save_dir, exist_ok=True)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{timestamp}.png"
        save_path = os.path.join(save_dir, filename)
        img.save(save_path)
        print(f"이미지가 Google Drive에 저장되었습니다: {save_path}")
    ## __save_to_drive(img: Image.Image, prompt: str)

    @staticmethod
    def __base64_to_pil(b64_string: str) -> Image.Image:
        img_bytes = base64.b64decode(b64_string)
        img = Image.open(io.BytesIO(img_bytes)).convert("RGB")
        return img
    ## __base64_to_pil(b64_string: str)

    @staticmethod
    def __pil_to_base64(img: Image.Image) -> str:
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        return base64.b64encode(buffered.getvalue()).decode()
    ## __pil_to_base64(img: Image.Image)
## class ImageAI

/usr/local/lib/python3.11/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.11/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segmen

**REST** **API**

- `generate` : 이미지 생성

In [ ]:
import os
from pyngrok import ngrok
from flask import Flask, render_template, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)
ngrok.set_auth_token(os.getenv('NGROK_AUTH_TOKEN'))

image_ai = ImageAI()

@app.route("/generate", methods=["POST"])
def generate():
    data = request.json
    base64_image = data.get("image")
    prompt = data.get("prompt")

    if not base64_image or not prompt:
        return jsonify({"error": "Missing image or prompt"}), 400

    try:
        result_base64 = image_ai.generate_from_sketch(base64_image, prompt)
        return jsonify({"image": result_base64})
    except Exception as e:
        return jsonify({"error": str(e)}), 500
## generate

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ControlNetHED.pth:   0%|          | 0.00/29.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(" * ngrok tunnel: ", public_url)
    app.run()

 * ngrok tunnel:  NgrokTunnel: "https://e6fb-34-125-247-244.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e6fb-34-125-247-244.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:44:28] "GET /read?limit=10 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:44:55] "GET /read?limit=10 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:45:25] "GET /read?limit=10 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:45:56] "GET /read?limit=10 HTTP/1.1" 200 -


  0%|          | 0/30 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:46:25] "GET /read?limit=10 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:46:42] "POST /generate HTTP/1.1" 200 -


이미지가 Google Drive에 저장되었습니다: /content/drive/MyDrive/20250701_044642.png


INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:46:55] "GET /read?limit=10 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:46:58] "POST /create HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:47:25] "GET /read?limit=10 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:47:26] "GET /read?limit=10 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Jul/2025 04:47:26] "GET /images/image_1751345207398.png HTTP/1.1" 404 -
